In [1]:
from os import walk
import pandas as pd

import numpy as np
from PIL import Image
from sklearn import preprocessing
import cv2
import os

In [2]:
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18445803828755482458
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6529366361
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9066601033619657632
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
with open("marks_train_small.txt", "r") as mt:
    marks_train = mt.readlines()

In [5]:
PIC_SIZE = (300, 300)

In [6]:
categories_train = {}
for line in marks_train:
    if line.startswith("#"):
        category = line.strip()[1:]
        paths = []
        marks = []
    else:
        path="train\\"+category+"\\"+line.split(";")[0]+".jpg"
        paths.append(path)
        markline = line.split(";")[1]
        markline = [int(i) for i in markline.split(",")]
        marks.append(markline)
    categories_train[category+"_path"] = paths
    categories_train[category+"_marks"] = marks

In [7]:
with open("marks_valid_small.txt", "r") as mv:
    marks_valid = mv.readlines()

In [8]:
categories_valid = {}
for line in marks_valid:
    if line.startswith("#"):
        category = line.strip()[1:]
        paths = []
        marks = []
    else:
        path="validation\\"+category+"\\"+line.split(";")[0]+".jpg"
        paths.append(path)
        markline = line.split(";")[1]
        markline = [int(i) for i in markline.split(",")]
        marks.append(markline)
    categories_valid[category+"_path"] = paths
    categories_valid[category+"_marks"] = marks

In [9]:
list(categories_train.keys())

['dress_path',
 'dress_marks',
 'hat_path',
 'hat_marks',
 'pants_path',
 'pants_marks',
 'shoes_path',
 'shoes_marks']

In [10]:
for category in ("hat", "shoes", "pants", "dress"):
    
    mtpaths = categories_train[f"{category}_path"]
    mtmarks = categories_train[f"{category}_marks"]

    mvpaths = categories_valid[f"{category}_path"]
    mvmarks = categories_valid[f"{category}_marks"]
    
    labels_train=[]
    for marks in mtmarks:
        x0, y0, deltaX, deltaY = marks
        x1 = x0 + deltaX
        y1 = y0 + deltaY
        labels_train.append([x0, y0, x1, y1])

    labels_valid=[]
    for marks in mvmarks:
        x0, y0, deltaX, deltaY = marks
        x1 = x0 + deltaX
        y1 = y0 + deltaY
        labels_valid.append([x0, y0, x1, y1])
        
    rel_labels_train = []
    for im, marks in zip(mtpaths, labels_train):
        height, width = cv2.imread(im, 0).shape
        x0, y0, x1, y1 = marks
        x0 /= width
        x1 /= width
        y0 /= height
        y1 /= height
        rel_labels_train.append([x0, y0, x1, y1])
    rel_labels_train = np.array(rel_labels_train)

    rel_labels_valid = []
    for im, marks in zip(mvpaths, labels_valid):
        height, width = cv2.imread(im, 0).shape
        x0, y0, x1, y1 = marks
        x0 /= width
        x1 /= width
        y0 /= height
        y1 /= height
        rel_labels_valid.append([x0, y0, x1, y1])
    rel_labels_valid = np.array(rel_labels_valid)
    
    gen_images = [file for file in mtpaths]
    bases = [cv2.imread(file) for file in gen_images]
    color_blurs = [cv2.GaussianBlur(file,(3,3),0) for file in bases]
    images = [cv2.resize(file, PIC_SIZE) for file in color_blurs]
    train_dataset = np.array(images)

    gen_images = [file for file in mvpaths]
    bases = [cv2.imread(file) for file in gen_images]
    color_blurs = [cv2.GaussianBlur(file,(3,3),0) for file in bases]
    images = [cv2.resize(file, PIC_SIZE) for file in color_blurs]
    valid_dataset = np.array(images)
    
    train_dataset_norm = train_dataset / 255.
    valid_dataset_norm = valid_dataset / 255.
    
    # make model
    input_tensor = Input(shape=(*PIC_SIZE, 3))

    vgg = VGG19(weights="imagenet", 
                include_top=False)(input_tensor)
    vgg.trainable = False

    flatten = Flatten()(vgg)
    bboxHead = Dense(2014, activation="relu")(flatten)
    bboxHead = Dense(128, activation="relu")(bboxHead)
    bboxHead = Dense(32, activation="relu")(bboxHead)
    bboxHead = Dense(4, activation="sigmoid")(bboxHead)

    model = Model(inputs=input_tensor, outputs=bboxHead)

    INIT_LR = 1e-4
    opt = Adam(INIT_LR)
    model.compile(loss="mae", optimizer=opt, metrics=['acc'])

    X_train = train_dataset_norm
    y_train = np.array(rel_labels_train) #labels_train #rel_labels_train
    X_test = valid_dataset_norm
    y_test = np.array(rel_labels_valid) #labels_valid #rel_labels_valid
    
    model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          epochs=50,
          #batch_size=32,
          verbose=2)
    
    #os.mkdir(f"{category}_model")
    
    model.save(f"{category}_model//bbox_{category}.h5")
    
    del model

Epoch 1/50
1/1 - 14s - loss: 0.1632 - acc: 0.5000 - val_loss: 0.1368 - val_acc: 0.4286
Epoch 2/50
1/1 - 1s - loss: 0.1286 - acc: 0.5000 - val_loss: 0.1378 - val_acc: 0.4286
Epoch 3/50
1/1 - 1s - loss: 0.1274 - acc: 0.5000 - val_loss: 0.1230 - val_acc: 0.5714
Epoch 4/50
1/1 - 1s - loss: 0.1151 - acc: 0.5000 - val_loss: 0.1035 - val_acc: 0.5714
Epoch 5/50
1/1 - 1s - loss: 0.1004 - acc: 0.5000 - val_loss: 0.0839 - val_acc: 0.5714
Epoch 6/50
1/1 - 1s - loss: 0.0835 - acc: 0.5000 - val_loss: 0.0681 - val_acc: 0.5714
Epoch 7/50
1/1 - 1s - loss: 0.0828 - acc: 0.5714 - val_loss: 0.0710 - val_acc: 0.4286
Epoch 8/50
1/1 - 1s - loss: 0.0803 - acc: 0.5000 - val_loss: 0.0648 - val_acc: 0.4286
Epoch 9/50
1/1 - 1s - loss: 0.0740 - acc: 0.5000 - val_loss: 0.0615 - val_acc: 0.5000
Epoch 10/50
1/1 - 1s - loss: 0.0695 - acc: 0.6071 - val_loss: 0.0884 - val_acc: 0.4286
Epoch 11/50
1/1 - 1s - loss: 0.0797 - acc: 0.6071 - val_loss: 0.0701 - val_acc: 0.5714
Epoch 12/50
1/1 - 1s - loss: 0.0708 - acc: 0.5000 -

Epoch 46/50
1/1 - 1s - loss: 0.1121 - acc: 0.6071 - val_loss: 0.0698 - val_acc: 0.6429
Epoch 47/50
1/1 - 1s - loss: 0.1062 - acc: 0.7143 - val_loss: 0.0691 - val_acc: 0.7143
Epoch 48/50
1/1 - 1s - loss: 0.1010 - acc: 0.7143 - val_loss: 0.0680 - val_acc: 0.7143
Epoch 49/50
1/1 - 1s - loss: 0.1006 - acc: 0.7143 - val_loss: 0.0663 - val_acc: 0.7857
Epoch 50/50
1/1 - 1s - loss: 0.1006 - acc: 0.7857 - val_loss: 0.0697 - val_acc: 0.7857
Epoch 1/50
1/1 - 2s - loss: 0.3752 - acc: 0.1786 - val_loss: 0.1339 - val_acc: 0.2143
Epoch 2/50
1/1 - 1s - loss: 0.1345 - acc: 0.3214 - val_loss: 0.2546 - val_acc: 0.7857
Epoch 3/50
1/1 - 1s - loss: 0.2643 - acc: 0.6786 - val_loss: 0.2094 - val_acc: 0.7857
Epoch 4/50
1/1 - 1s - loss: 0.2194 - acc: 0.6786 - val_loss: 0.1511 - val_acc: 0.7857
Epoch 5/50
1/1 - 1s - loss: 0.1592 - acc: 0.6786 - val_loss: 0.1295 - val_acc: 0.7857
Epoch 6/50
1/1 - 1s - loss: 0.1378 - acc: 0.6786 - val_loss: 0.1007 - val_acc: 0.7857
Epoch 7/50
1/1 - 1s - loss: 0.1135 - acc: 0.6786 

Epoch 41/50
1/1 - 1s - loss: 0.0234 - acc: 0.9643 - val_loss: 0.0672 - val_acc: 0.3571
Epoch 42/50
1/1 - 1s - loss: 0.0213 - acc: 0.9643 - val_loss: 0.0668 - val_acc: 0.3571
Epoch 43/50
1/1 - 1s - loss: 0.0203 - acc: 1.0000 - val_loss: 0.0664 - val_acc: 0.3571
Epoch 44/50
1/1 - 1s - loss: 0.0224 - acc: 1.0000 - val_loss: 0.0674 - val_acc: 0.3571
Epoch 45/50
1/1 - 1s - loss: 0.0191 - acc: 1.0000 - val_loss: 0.0672 - val_acc: 0.3571
Epoch 46/50
1/1 - 1s - loss: 0.0171 - acc: 1.0000 - val_loss: 0.0676 - val_acc: 0.3571
Epoch 47/50
1/1 - 1s - loss: 0.0173 - acc: 1.0000 - val_loss: 0.0693 - val_acc: 0.5000
Epoch 48/50
1/1 - 1s - loss: 0.0178 - acc: 1.0000 - val_loss: 0.0700 - val_acc: 0.4286
Epoch 49/50
1/1 - 1s - loss: 0.0146 - acc: 0.9643 - val_loss: 0.0683 - val_acc: 0.4286
Epoch 50/50
1/1 - 1s - loss: 0.0159 - acc: 1.0000 - val_loss: 0.0701 - val_acc: 0.4286
